In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Conv2D, Input, Dense, MaxPool2D, BatchNormalization, GlobalAvgPool2D, Flatten
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# for callback
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import cv2 as cv

In [2]:
image = cv.imread("./DatasetMedian/Training/MildDemented/mildDem0.jpg")
image.shape

(208, 176, 3)

In [3]:
def irm_model(nbr_classes) :
    my_input = Input(shape=(208, 176, 3)) # taille moyenne et les 3 canaux RGB
    x= Conv2D(32, (3,3), activation='relu')(my_input)
    x= Conv2D(64, (3,3), activation='relu')(x)
    x= MaxPool2D()(x)
    x= BatchNormalization()(x)

    x= Conv2D(128, (3,3), activation='relu')(x)
    x= MaxPool2D()(x)
    x= BatchNormalization()(x)

    x= GlobalAvgPool2D()(x)
    #x = Flatten()(x)
    x= Dense(64, activation='relu')(x)
    x= Dense(nbr_classes, activation='softmax')(x)
    model = Model(inputs=my_input, outputs=x)
    return model

In [4]:
if __name__=='__main__' :

    model = irm_model(43)
    model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 208, 176, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 206, 174, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 204, 172, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 102, 86, 64)      0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 102, 86, 64)      256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 100, 84, 128)      73856 

In [5]:
def create_generators(batch_size, train_data_path, val_data_path, test_data_path) :
    preprocessor = ImageDataGenerator(
        rescale = 1/255. # pour assurer une division flottante
    )

    # !très important pour prendre des données en prenant chaque sous dossier comme classe à part entière
    train_generator = preprocessor.flow_from_directory(
        train_data_path,
        class_mode="categorical",
        target_size=(60,60), #resize all images
        color_mode = 'rgb', # type d'images
        shuffle = True,
        batch_size=batch_size
    )

    val_generator = preprocessor.flow_from_directory(
        val_data_path,
        class_mode="categorical",
        target_size=(60,60), #resize all images
        color_mode = 'rgb', # type d'images
        shuffle = False,
        batch_size=batch_size
    )

    test_generator = preprocessor.flow_from_directory(
        test_data_path,
        class_mode="categorical",
        target_size=(60,60), #resize all images
        color_mode = 'rgb', # type d'images
        shuffle = False,
        batch_size=batch_size
    )

    return train_generator, val_generator, test_generator

In [6]:
train_data_path = "./DatasetMedian/Training"
val_data_path = "./DatasetMedian/Validation"
test_data_path = "./DatasetMedian/test"
batch_size = 16

train_generator, val_generator, test_generator= create_generators(batch_size, train_data_path, val_data_path, test_data_path)

nbr_classes = train_generator.num_classes

Found 4607 images belonging to 4 classes.
Found 514 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [7]:
model = irm_model(nbr_classes)

In [8]:
# callbacks
path_to_save_model = './Models'
ckpt_saver = ModelCheckpoint(
    path_to_save_model,
    monitor='val_accuracy', # sur quoi on se base pour voir le meilleur
    mode = 'max', # max de l'accuracy sur la validation
    save_best_only = True,
    save_freq='epoch', # ne voit qu'à la fin de l'époque
    verbose=1
) 

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=10 # après 10 époques ca change pas on s'arrête
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # on choisit categorical_crossentropy car dans les générateurs on a défini categorical comme class_mode

epochs = 50

In [9]:
model.fit(
    train_generator,
    epochs = epochs,
    batch_size = batch_size,
    validation_data = val_generator,
    callbacks=[ckpt_saver]
)

Epoch 1/50
288/288 [==============================] - ETA: 0s - loss: 0.9360 - accuracy: 0.5702
Epoch 1: val_accuracy improved from -inf to 0.49805, saving model to .\Models
INFO:tensorflow:Assets written to: .\Models\assets
288/288 [==============================] - 85s 286ms/step - loss: 0.9360 - accuracy: 0.5702 - val_loss: 1.1784 - val_accuracy: 0.4981
Epoch 2/50
288/288 [==============================] - ETA: 0s - loss: 0.8418 - accuracy: 0.6132
Epoch 2: val_accuracy did not improve from 0.49805
288/288 [==============================] - 72s 248ms/step - loss: 0.8418 - accuracy: 0.6132 - val_loss: 3.9748 - val_accuracy: 0.4981
Epoch 3/50
288/288 [==============================] - ETA: 0s - loss: 0.7615 - accuracy: 0.6629
Epoch 3: val_accuracy improved from 0.49805 to 0.52140, saving model to .\Models
INFO:tensorflow:Assets written to: .\Models\assets
288/288 [==============================] - 64s 223ms/step - loss: 0.7615 - accuracy: 0.6629 - val_loss: 1.4463 - val_accuracy: 0.521

In [ ]:
model.evaluate(test_generator, batch_size=16)

80/80 [==============================] - 7s 87ms/step - loss: 3.3109 - accuracy: 0.5880


[3.310868263244629, 0.587959349155426]